In [ ]:
import numpy as np
import torch
import cv2
from sklearn.metrics import mean_squared_error
from regressor import CNNModel
from loader import resize

def load_model(model_path='model.pth'):
    model = CNNModel()
    model.load_state_dict(torch.load(model_path))
    model.eval()
    return model

def preprocess_image(image, size=(224, 224)):
    resized_image, _ = resize(image, np.zeros(8), size)
    resized_image = torch.tensor(resized_image, dtype=torch.float32).permute(2, 0, 1) / 255.0
    return resized_image.unsqueeze(0)

def de_normalize_points(points, original_size, resized_size=(224, 224)):
    original_width, original_height = original_size
    resized_width, resized_height = resized_size

    scale_x = original_width / resized_width
    scale_y = original_height / resized_height

    de_normalized_points = points.copy()
    for i in range(4):
        de_normalized_points[2*i] *= scale_x
        de_normalized_points[2*i+1] *= scale_y

    return de_normalized_points

def corners(image: np.ndarray) -> np.ndarray:
    model = load_model()
    original_size = image.shape[1], image.shape[0]  # width, height

    preprocessed_image = preprocess_image(image)
    with torch.no_grad():
        output_points = model(preprocessed_image).squeeze(0).numpy()

    de_normalized_points = de_normalize_points(output_points, original_size)
    return de_normalized_points

def evaluate_model(data_dir, model_path='model.pth'):
    from loader import load  # Ensure load function is correctly imported here
    
    images, true_points = load(data_dir)
    model = load_model(model_path)

    predicted_points = []
    for image in images:
        predicted_points.append(corners(image))

    predicted_points = np.array(predicted_points)
    true_points = np.array(true_points)

    mse = mean_squared_error(true_points.reshape(-1), predicted_points.reshape(-1))
    print(f"Mean Squared Error: {mse:.4f}")

if __name__ == '__main__':
    image_path = 'path_to_some_image.jpg'  # Replace with an actual image path for testing
    image = cv2.imread(image_path)
    if image is not None:
        predicted_corners = corners(image)
        print(f"Predicted corners: {predicted_corners}")
    else:
        print("Image not found or unable to load.")
    
    data_dir = '/home/navid/Desktop/VisionProject/four-corners'
    evaluate_model(data_dir)
